In [ ]:
# Jupyter config
%load_ext rpy2.ipython
%matplotlib inline
%config InlineBackend.figure_format = 'svg'  # Or 'retina'

In [ ]:
# Python imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import *
from sklearn.mixture import *

#plt.style.use('seaborn-whitegrid')  # Set the aesthetic style of the plots